In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import pandas as pd
import urllib.request
import numpy as np
import matplotlib as plt
from IPython.display import display
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import scipy.sparse as sp
from sklearn import metrics

In [ ]:
# read in the file and convert to sparse matrix
cf = open('pancancer_mutations_merged.csv', 'r')
pat_out = OrderedDict()
hugos = dict()
hugoidx = 0
cols = cf.readline().split(",")
labels_d = OrderedDict()
for i, line in enumerate(cf):
  nl = dict(zip(cols, line.split(',')))
  pat, hugo, ct = nl['bcr_patient_barcode'], nl['Hugo_Symbol'], nl['cancer_type']
  labels_d[pat] = ct
  if hugo not in hugos:
    hugos[hugo] = hugoidx
    hugoidx +=1
  if pat not in pat_out:
    pat_out[pat] = []
  pat_out[pat].append(hugos[hugo])


In [ ]:
mat = sp.dok_matrix((len(pat_out.keys()), len(hugos)), dtype=np.int8)

In [ ]:
for i, (patient, hugos) in enumerate(pat_out.items()):
  mat[i, hugos] = 1
mat = mat.tocsr()

In [ ]:
#from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=200)
principalComponents = svd.fit_transform(mat)


In [ ]:
labels = pd.DataFrame(labels_d.values())
shuffle = np.random.permutation(np.arange(mat.shape[0]))
X, Y  = principalComponents[shuffle], labels.iloc[shuffle]
#X, Y  = mat[shuffle], labels.iloc[shuffle]

In [ ]:
train_X = X[:6000]
train_Y = Y[:6000]

test_X = X[6000:]
test_Y = Y[6000:]

In [ ]:
clf = LogisticRegression()

# Train Decision Tree Classifer
clf = clf.fit(train_X, train_Y)

In [ ]:
pred_Y = clf.predict(test_X)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_Y, pred_Y))

In [ ]:
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(train_X, train_Y)
pred_Y = clf.predict(test_X)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_Y, pred_Y))

In [ ]:
clf = RandomForestClassifier(n_estimators=500)

# Train Decision Tree Classifer
clf = clf.fit(train_X, train_Y)
pred_Y = clf.predict(test_X)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_Y, pred_Y))